In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm_notebook as tqdm
from IPython.display import display

import re
import pymorphy2
import nltk
from nltk.corpus import stopwords
import joblib

from sklearn.feature_extraction.text import TfidfVectorizer
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score

import warnings
warnings.filterwarnings('ignore')

pd.set_option('max_colwidth', 100)

In [ ]:
train = pd.read_csv('../data/train.csv')
val = pd.read_csv('../data/val.csv')

train.shape, val.shape

In [4]:
from custom.text import TextPreprocessor
from custom.predict import ModelBoosting

Основной упор был сделан на корректную предобработку текста для извлечения максимальной информативности. Предварительно вручную были проанализированы текста с положительными метками, чтобы понять, как авторы объявлений склонны размещать свои контакты в тексте. Было замечено, что многие пытаются хитрым образом замаскировать текст с помощью промежуточных символов или же номером в форме полного текста. Функция ниже позволяет явно выделять данные номера и переводить их в "спецстроки", которые определяются длиной последовательности цифр и начальной цифрой.

Также были рассмотрены тексты с сайтами, никами инстаграма/телеграмма, электронной почтой. Эти случаи также были учтены. В виду достаточно затратного по времени применения сложных регулярных выражений к строкам, функции замен были сильно упрощены к концу разработки.

In [5]:
preprocess = TextPreprocessor(max_len=400)

Любая последовательность цифр приводится к единой строке, определяющейся двумя параметрами: первая цифра, количество цифр. Таким образом удается перевести номера телефонов в один из случаев: <7_digits_11>, <8_digits_11>, <9_digits_10> (это только мобильные номера, функция также позволяет определять и номера, состоящие из 6 цифр). 

Текст ограничbвается с каждой стороны по 200 слов. Гипотеза в следующем: ключевую информацию для связи с автором обычно не оставляют в середине объявления. Как правило, если текст большой, в середине содержится объемная информация о предлагаемых товарах и услугах. Данный подход позволяет немного снизить скорость предобработки. 

In [ ]:
train = preprocess.full_text_preprocessing(train)

In [7]:
%%time
val = preprocess.full_text_preprocessing(val)

CPU times: user 14.7 s, sys: 38.8 ms, total: 14.7 s
Wall time: 14.7 s


In [8]:
train.to_csv('../../data_dev/train_preproc.csv',index=False)
val.to_csv('../../data_dev/val_preproc.csv',index=False)

train = pd.read_csv('../../data_dev/train_preproc.csv',lineterminator='\n')
val = pd.read_csv('../../data_dev/val_preproc.csv',lineterminator='\n')

In [11]:
train['normalized_text'] = train['normalized_text'].fillna('')
val['normalized_text'] = val['normalized_text'].fillna('')

Базовая модель является объединением Tfidf преобразования и бустинга lightgbm. Дополнительно можно было бы рассмотреть подбор гиперпараметров и отбор признаков, но в данной итерации это было пропущено.

In [12]:
from sklearn.model_selection import train_test_split
train, train_val = train_test_split(train, test_size=0.2, random_state=42)

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
from lightgbm import LGBMClassifier

tfidf = TfidfVectorizer(max_features=2000)
lgbm = LGBMClassifier(random_state=42, n_estimators=5000, n_jobs=-1)

train_tfidf = tfidf.fit_transform(train['normalized_text'])
train_val_tfidf = tfidf.transform(train_val['normalized_text'])
val_tfidf = tfidf.transform(val['normalized_text'])

In [15]:
lgbm.fit(train_tfidf, train.is_bad, eval_set=[(train_val_tfidf, train_val.is_bad)],
         early_stopping_rounds=50, eval_metric='auc', verbose=250)

[250]	valid_0's auc: 0.963624	valid_0's binary_logloss: 0.176687
[500]	valid_0's auc: 0.96633	valid_0's binary_logloss: 0.16974
[750]	valid_0's auc: 0.967646	valid_0's binary_logloss: 0.166165
[1000]	valid_0's auc: 0.968473	valid_0's binary_logloss: 0.163689
[1250]	valid_0's auc: 0.969099	valid_0's binary_logloss: 0.161982
[1500]	valid_0's auc: 0.969543	valid_0's binary_logloss: 0.160603
[1750]	valid_0's auc: 0.969896	valid_0's binary_logloss: 0.159566
[2000]	valid_0's auc: 0.970163	valid_0's binary_logloss: 0.15869
[2250]	valid_0's auc: 0.970347	valid_0's binary_logloss: 0.157972
[2500]	valid_0's auc: 0.97052	valid_0's binary_logloss: 0.1573
[2750]	valid_0's auc: 0.970625	valid_0's binary_logloss: 0.156864
[3000]	valid_0's auc: 0.970774	valid_0's binary_logloss: 0.156408
[3250]	valid_0's auc: 0.970906	valid_0's binary_logloss: 0.156118


LGBMClassifier(n_estimators=5000, random_state=42)

In [18]:
(roc_auc_score(train.is_bad, lgbm.predict_proba(train_tfidf).T[1]),
 roc_auc_score(train_val.is_bad, lgbm.predict_proba(train_val_tfidf).T[1]),
 roc_auc_score(val.is_bad, lgbm.predict_proba(val_tfidf).T[1]))

(0.99375236477077, 0.9709343641279251, 0.9596928118254923)

In [16]:
s = pd.Series(dict(zip(tfidf.get_feature_names(), lgbm.feature_importances_))).sort_values(ascending=False)

In [17]:
s.head(20)

9_digits_10    1241
звонить        1044
8_digits_11    1010
состояние       843
телефон         798
размер          762
новый           702
номер           664
тело            559
продать         558
цена            556
ru              542
3_digits_1      502
работа          422
писать          402
9_digits_3      400
хороший         399
сайт            393
наличие         388
продавать       383
dtype: int32

In [20]:
val['score'] = lgbm.predict_proba(val_tfidf).T[1]

In [22]:
from sklearn.metrics import precision_score, recall_score
thr=0.65
precision_score(val.is_bad, val.score>thr), recall_score(val.is_bad, val.score>thr)

(0.9009174311926605, 0.7400150715900528)

In [23]:
val.sort_values(by='score', ascending=False).head(int(len(val)*0.1)).is_bad.mean()

0.9642637091805298

# Вторая задача

При попытке решить вторую задачу стало понятно, что подход с т.н. полуручной обработкой текста не покрывает все возможные варианты. Трудности возникли именно из-за большого количества функций: полностью теряется возможная 1-1 связь между исходными и конечными текстами. 

На самом деле, около трети проблемных объявлений можно выделить с IoU --> 1. Достаточно часто контакт оставляют в самом конце текста. У нас уже есть инфрмация о том, какой тип номера представлен в обработанном тексте, нужно просто правильно подсчитать отступы с учетом имющихся промежуточных символов, не являющихся цифрами.

Начало возможной функции представлено ниже.

In [ ]:
def find_idx(df):
    df['first_idx'] = None
    df['last_idx'] = None
    df['len_str_txt'] = df.text_w_spaces.str.len()
    ends_phone = df[df.normalized_text.str.endswith('<phone>')]
    ends_8_digits_11 = df[df.normalized_text.str.endswith('<8_digits_11>')]
    ends_7_digits_11 = df[df.normalized_text.str.endswith('<7_digits_11>')]
    ends_9_digits_10 = df[df.normalized_text.str.endswith('<9_digits_10>')]
    
    ends_phone['last_idx'] = ends_phone.len_str_txt-1 #не учитываются точки в конец и тд
    ends_phone['first_idx'] = ends_phone.len_str_txt-13
    
    ends_8_digits_11['last_idx'] = ends_phone.len_str_txt-1 #не учитываются точки в конец и тд
    ends_8_digits_11['first_idx'] = ends_phone.len_str_txt-13
    

In [24]:
val_bad = val[val.is_bad==1]
val_bad.shape

(3981, 14)

In [25]:

val_bad[val_bad.normalized_text.str.endswith('<phone>') |\
    val_bad.normalized_text.str.endswith('<8_digits_11>') |\
    val_bad.normalized_text.str.endswith('<7_digits_11>') |\
    val_bad.normalized_text.str.endswith('<9_digits_10>')][['text_w_spaces', 'normalized_text', 'score']].shape

(1690, 3)

In [26]:
1690/3981

0.42451645315247427

Однако все же такой данный подход, в силу ограничения по времени, было решено оставить. 

На текущий момент решение второй задачи на возможной второй попытке видится следующим:
* Не делать сильный упор на предобработку, а построить НС (здесь больше видится сверточная 1d, так как понимание контекста из прошлого в текста здесь кажется не таким важным и хорошие результаты по Tfidf преобразованию это подтвердили). Примерные наработки на предобработанном тексте представлены в ноутбуках cnn.ipynb, rnn.ipynb.
* Использовать Subwords Tokenization, чтобы погрешность при определении границ была ниже. Еще один аргумент за данный подход: низкая предиктивная способность сеток на тех же токенах (представлено в ноутбуках) по сравнению с бустингом. Возможно из-за искуственного сокращения информации на этапе предобработки сетям оказалось недостаточно просто факта наличия ключевых токенов, и нужно дать больше вариативности на вход моделям для устойчивого обучения.
* После обучения модели применить алгоритмы библиотеки captum (или других библиотек интерпретации) для выделения "горячих" зон предложения. Данные зоны и будут определять потенциальный текст контакта.

Как ожидаемый итог, потенциальные зоны позволят в среднем получить приемлемый IoU, и при всем при этом попутно решится задача бинарной классификации.